In [14]:
import numpy as np
import pandas as pd
#import scipy
from scipy.special import binom
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
# from scipy.stats import quantile
# from itertools import combinations

# Load source_ACR module
# Assuming the relevant functions are defined in source_ACR_py.py
import source_ACR_py as acr

In [13]:
# Initialization
thres_acr = 0.01
thres_positive = 0.01
thres_unbounded = 0.1
num_S = 2
max_order = 2
num_R = 3
source_mat = None
product_mat = None
stoi_mat = None

num_total_C = int(binom(max_order + num_S, num_S))
total_complex = acr.make_total_complexes(num_S, max_order)

num_total_R = num_total_C * (num_total_C - 1)
num_total_net = int(binom(num_total_R, num_R))

num_repeat_net = num_total_net
num_repeat_par = 5
num_repeat_init = 20

In [6]:
# Beginning of the squential (or random) network search
matrix_R_id = np.zeros((num_R, num_repeat_net), dtype=int)
list_acr_id = np.zeros((num_S, num_repeat_net), dtype=int)
list_unbnd_id = np.zeros((num_S, num_repeat_net), dtype=int)

rnd_search = False
#for net_id in range(1000, 1002):
for net_id in range(1, num_total_net+1):
    if net_id % 10 == 1:
        print(f"======= Analyzing network {net_id:4} =======")

    if rnd_search:
        list_R_id = np.random.choice(range(1, num_total_R + 1), num_R, replace=False)
    else:
        list_R_id = acr.net_id_to_R_id_list(net_id, num_total_R, num_R)

    list_source_id, list_product_id = zip(*[acr.reaction_id_to_complex_id(R_id, num_total_C) for R_id in list_R_id])
    source_mat = total_complex[:, np.array(list_source_id)-1]
    product_mat = total_complex[:, np.array(list_product_id)-1]
    stoi_mat = product_mat - source_mat

    # Simulation settings
    #kappa1 = np.random.uniform(low=0.0, high=10.0, size=num_R)
    #x_init = np.random.uniform(low=1.0, high=20.0, size=num_S)
    tspan1 = (0.0, 50.0)

    ub_param = 10 * np.ones(num_R) # Lower bounds for the parameter values 
    lb_param = np.zeros(num_R) # Upper bounds for the parameter values

    ub_init = 20 * np.ones(num_S) # Lower bounds for the initial values
    lb_init = np.ones(num_S) # Upper bounds for the initial values

    list_acr_id_par = np.zeros((num_S, num_repeat_par), dtype=int)
    list_unbnd_id_par = np.zeros((num_S, num_repeat_par), dtype=int)

    for iter_par in range(num_repeat_par):
        kappa1 = np.random.uniform(low=lb_param, high=ub_param, size=num_R)
        final_val_mat = np.zeros((num_S, num_repeat_init))
        latter_val_mat = np.zeros((num_S, num_repeat_init))

        for iter_init in range(1, num_repeat_init + 1):
            x_init = np.random.uniform(low=lb_init, high=ub_init, size=num_S)
            t_eval = np.arange(tspan1[0], tspan1[1]+1, 1)
            sol1 = solve_ivp(acr.MAK_rescaled, tspan1, x_init, args = (kappa1, source_mat, stoi_mat), method='RK45', t_eval=t_eval)
            sol_mat1 = sol1.y.T
            length_tspan = sol_mat1.shape[0]
            final_val_mat[:, iter_init - 1] = sol_mat1[length_tspan - 1, :]

            if int(np.floor(0.8 * length_tspan)) >= 1:
                latter_val_mat[:, iter_init - 1] = sol_mat1[int(np.floor(0.8 * length_tspan)), :]

        upp_val = np.array([np.quantile(final_val_mat[i, :], 0.6) for i in range(num_S)])
        low_val = np.array([np.quantile(final_val_mat[i, :], 0.4) for i in range(num_S)])

        for i in range(num_S):
            if upp_val[i] - low_val[i] < thres_acr and low_val[i] > thres_positive:
                list_acr_id_par[i, iter_par] = 1
            if np.mean(latter_val_mat[i, :]) > lb_init[i] and np.mean(final_val_mat[i, :]) > np.mean(latter_val_mat[i, :]) + thres_unbounded:
                list_unbnd_id_par[i, iter_par] = 1

    for i in range(num_S):
        if np.sum(list_acr_id_par[i, :]) > 0:
            list_acr_id[i, net_id - 1] += 1
        if np.prod(list_acr_id_par[i, :]) > 0:
            list_acr_id[i, net_id - 1] += 1
        if np.sum(list_unbnd_id_par[i, :]) > 0:
            list_unbnd_id[i, net_id - 1] += 1
        if np.prod(list_unbnd_id_par[i, :]) > 0:
            list_unbnd_id[i, net_id - 1] += 1

    matrix_R_id[:, net_id - 1] = list_R_id

======= Analyzing network    1 =======
======= Analyzing network   11 =======
======= Analyzing network   21 =======
======= Analyzing network   31 =======
======= Analyzing network   41 =======
======= Analyzing network   51 =======
======= Analyzing network   61 =======
======= Analyzing network   71 =======
======= Analyzing network   81 =======
======= Analyzing network   91 =======
======= Analyzing network  101 =======
======= Analyzing network  111 =======
======= Analyzing network  121 =======
======= Analyzing network  131 =======
======= Analyzing network  141 =======
======= Analyzing network  151 =======
======= Analyzing network  161 =======
======= Analyzing network  171 =======
======= Analyzing network  181 =======
======= Analyzing network  191 =======
======= Analyzing network  201 =======
======= Analyzing network  211 =======
======= Analyzing network  221 =======
======= Analyzing network  231 =======
======= Analyzing network  241 =======
======= Analyzing network

In [7]:
# Convert arrays to pandas DataFrames and save to CSV files
# We take the transpose to make the data fit the Excel program capacity.
list_acr_id_df = pd.DataFrame(list_acr_id).transpose()
list_unbnd_id_df = pd.DataFrame(list_unbnd_id).transpose()
matrix_R_id_df = pd.DataFrame(matrix_R_id).transpose()

path = "/Users/hyukpyohong/Dropbox/CRN_dynamicACR/Codes/data/"
list_acr_id_df.to_csv(path + f"python_list_acr_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)
list_unbnd_id_df.to_csv(path + f"python_list_unbnd_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)
matrix_R_id_df.to_csv(path + f"python_matrix_R_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)

In [ ]:
### ==========  The code below is just temporary test code  ============